In [ ]:
import os
import zipfile
import tempfile
import random
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pywintypes
import win32file
import win32con
import win32com.client

# Utwórz folder, jeśli nie istnieje
folder = r"C:\Users\a\Desktop\directory_with_changed_files"
os.makedirs(folder, exist_ok=True)

def losowa_data_po_1_kwietnia_2024():
    start = datetime(2024, 4, 1)  # poprawny miesiąc
    end = datetime.now()
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def format_data_iso(dt):
    return dt.isoformat(timespec='seconds') + "Z"

def ustaw_systemowa_date_pliku(file_path, utworzono_dt):
    zmodyfikowano_dt = datetime.now() - timedelta(days=24)

    wintime_utw = pywintypes.Time(utworzono_dt)
    wintime_mod = pywintypes.Time(zmodyfikowano_dt)

    handle = win32file.CreateFile(
        file_path, win32con.GENERIC_WRITE, 0, None,
        win32con.OPEN_EXISTING, win32con.FILE_ATTRIBUTE_NORMAL, None)
    
    win32file.SetFileTime(handle, wintime_utw, None, wintime_mod)
    handle.close()

def modyfikuj_corexml(path, nowy_czas):
    ns = {
        'cp': 'http://schemas.openxmlformats.org/package/2006/metadata/core-properties',
        'dc': 'http://purl.org/dc/elements/1.1/',
        'dcterms': 'http://purl.org/dc/terms/',
        'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
    }
    tree = ET.parse(path)
    root = tree.getroot()
    if (el := root.find('dc:creator', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('cp:lastModifiedBy', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('dcterms:created', ns)) is not None:
        el.text = format_data_iso(nowy_czas)
    if (el := root.find('dcterms:modified', ns)) is not None:
        el.text = format_data_iso(nowy_czas)
    tree.write(path, encoding='utf-8', xml_declaration=True)

def modyfikuj_appxml(path):
    try:
        tree = ET.parse(path)
        root = tree.getroot()
        for tag in ['Company', 'Manager', 'Application']:
            el = root.find(tag)
            if el is not None:
                el.text = "Brak"
        tree.write(path, encoding='utf-8', xml_declaration=True)
    except:
        pass

def przetworz_xlsx(file_path, output_path, nowa_data):
    with tempfile.TemporaryDirectory() as tempdir:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(tempdir)

        core_path = os.path.join(tempdir, 'docProps', 'core.xml')
        app_path = os.path.join(tempdir, 'docProps', 'app.xml')

        if os.path.exists(core_path):
            modyfikuj_corexml(core_path, nowa_data)
        if os.path.exists(app_path):
            modyfikuj_appxml(app_path)

        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as new_zip:
            for foldername, subfolders, filenames in os.walk(tempdir):
                for filename in filenames:
                    full_path = os.path.join(foldername, filename)
                    rel_path = os.path.relpath(full_path, tempdir)
                    new_zip.write(full_path, rel_path)

    ustaw_systemowa_date_pliku(output_path, nowa_data)

def konwertuj_xls_na_xlsx(excel, file_path):
    output_path = file_path + "x"  # dodaje 'x' = .xlsx
    wb = excel.Workbooks.Open(file_path)
    wb.SaveAs(output_path, FileFormat=51)
    wb.Close()
    return output_path

def przetworz_folder(folder_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False

    for fname in os.listdir(folder_path):
        full_path = os.path.join(folder_path, fname)

        if fname.endswith(".xls") and not fname.endswith(".xlsx"):
            try:
                print(f"🔄 Konwertuję .xls → .xlsx: {fname}")
                full_path = konwertuj_xls_na_xlsx(excel, full_path)
                fname = os.path.basename(full_path)
            except Exception as e:
                print(f"⚠️ Błąd konwersji: {fname} → {e}")
                continue

        if fname.endswith(".xlsx") and not fname.endswith("_random_date.xlsx"):
            nowa_data = losowa_data_po_1_kwietnia_2024()
            output_path = os.path.join(folder_path, fname.replace(".xlsx", "_random_date.xlsx"))
            try:
                przetworz_xlsx(full_path, output_path, nowa_data)
                print(f"✅ {fname} → {os.path.basename(output_path)} | Utworzono: {nowa_data.date()} | Zmodyfikowano: {(datetime.now() - timedelta(days=24)).date()}")
            except Exception as e:
                print(f"❌ Błąd przetwarzania {fname}: {e}")

    excel.Quit()

# 🔁 Start przetwarzania
przetworz_folder(folder)


✅ Wzór.xlsx → Wzór_random_date.xlsx | Utworzono: 2024-07-23 | Zmodyfikowano: 2025-06-19
✅ Wzór1.xlsx → Wzór1_random_date.xlsx | Utworzono: 2025-02-27 | Zmodyfikowano: 2025-06-19
✅ Wzór12.xlsx → Wzór12_random_date.xlsx | Utworzono: 2024-10-04 | Zmodyfikowano: 2025-06-19
✅ Wzór122.xlsx → Wzór122_random_date.xlsx | Utworzono: 2025-04-16 | Zmodyfikowano: 2025-06-19
